In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

# --- Load your CSV dataset ---
df = pd.read_csv("/content/SCOA_A4.csv")

# Separate features and target
X = df.drop("species", axis=1).values
y = LabelEncoder().fit_transform(df["species"].values)  # convert labels to integers

# --- Genetic Algorithm Setup ---
POP_SIZE = 20       # number of individuals
N_GENERATIONS = 10  # number of generations
MUTATION_RATE = 0.2

# Chromosome: [max_depth, min_samples_split]
def create_chromosome():
    return [random.randint(1, 20), random.randint(2, 10)]

def fitness(chromosome):
    max_depth, min_samples_split = chromosome
    model = DecisionTreeClassifier(max_depth=max_depth,
                                   min_samples_split=min_samples_split,
                                   random_state=42)
    scores = cross_val_score(model, X, y, cv=5)
    return scores.mean()

def selection(population, fitnesses):
    # Select the top 2 individuals
    idx = np.argsort(fitnesses)[-2:]
    return [population[idx[0]], population[idx[1]]]

def crossover(parent1, parent2):
    # Single point crossover
    point = random.randint(1, len(parent1)-1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(chromosome):
    if random.random() < MUTATION_RATE:
        chromosome[0] = random.randint(1, 20)
    if random.random() < MUTATION_RATE:
        chromosome[1] = random.randint(2, 10)
    return chromosome

# --- Initialize population ---
population = [create_chromosome() for _ in range(POP_SIZE)]

# --- Run GA ---
for gen in range(N_GENERATIONS):
    fitnesses = [fitness(chromo) for chromo in population]
    print(f"Generation {gen} - Best Fitness: {max(fitnesses):.4f}")

    new_population = []
    parents = selection(population, fitnesses)

    # Generate next generation
    for _ in range(POP_SIZE // 2):
        child1, child2 = crossover(parents[0], parents[1])
        new_population.append(mutate(child1))
        new_population.append(mutate(child2))

    population = new_population

# --- Best hyperparameters ---
fitnesses = [fitness(chromo) for chromo in population]
best_idx = np.argmax(fitnesses)
print("\nBest Hyperparameters:", population[best_idx])
print("Best Accuracy:", fitnesses[best_idx])

Generation 0 - Best Fitness: 0.9733
Generation 1 - Best Fitness: 0.9733
Generation 2 - Best Fitness: 0.9733
Generation 3 - Best Fitness: 0.9733
Generation 4 - Best Fitness: 0.9733
Generation 5 - Best Fitness: 0.9733
Generation 6 - Best Fitness: 0.9733
Generation 7 - Best Fitness: 0.9733
Generation 8 - Best Fitness: 0.9733
Generation 9 - Best Fitness: 0.9733

Best Hyperparameters: [3, 9]
Best Accuracy: 0.9733333333333334


**Genetic Algorithm (GA)** to optimize **Decision Tree hyperparameters** (specifically `max_depth` and `min_samples_split`) using a dataset called `SCOA_A4.csv`.

The goal of this program:

> Use a **Genetic Algorithm (GA)** to find the best Decision Tree parameters that yield the **highest classification accuracy** on your dataset.

We’re tuning:

* `max_depth`: how deep the decision tree can grow.
* `min_samples_split`: the minimum number of samples required to split a node.

---

Step 1: Import Libraries

```python
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
```

Explanation:

* `numpy` → used for numerical operations (sorting, arrays).
* `pandas` → used to read and manipulate your CSV dataset.
* `random` → used for random number generation (essential in GA).
* `cross_val_score` → performs cross-validation to measure model accuracy.
* `DecisionTreeClassifier` → your machine learning model.
* `LabelEncoder` → converts categorical labels (like species names) into numeric values.

---

Step 2: Load and Prepare the Dataset

```python
df = pd.read_csv("/content/SCOA_A4.csv")
```

Loads your dataset from Google Colab or local path.
Example dataset might look like:

| sepal_length | sepal_width | petal_length | petal_width | species    |
| ------------ | ----------- | ------------ | ----------- | ---------- |
| 5.1          | 3.5         | 1.4          | 0.2         | setosa     |
| 6.0          | 3.4         | 4.5          | 1.6         | versicolor |

---

Split into Features (X) and Target (y)

```python
X = df.drop("species", axis=1).values
y = LabelEncoder().fit_transform(df["species"].values)
```

* `X`: input features (numerical columns except species)
* `y`: encoded class labels (`species` column converted to integers)

  * Example: setosa → 0, versicolor → 1, virginica → 2

---

Step 3: GA Hyperparameters

```python
POP_SIZE = 20       # number of chromosomes (solutions)
N_GENERATIONS = 10  # evolution cycles
MUTATION_RATE = 0.2 # 20% chance for mutation
```

* Each **chromosome** = one possible set of Decision Tree hyperparameters.
* We evolve the population for **10 generations**, improving accuracy.
* **Mutation rate 0.2** means: 20% chance that a chromosome gene mutates (changes randomly).

---

Step 4: Chromosome Definition

```python
def create_chromosome():
    return [random.randint(1, 20), random.randint(2, 10)]
```

Meaning:

Each chromosome = `[max_depth, min_samples_split]`

* `max_depth` → between 1 and 20
* `min_samples_split` → between 2 and 10

So examples:

```
[3, 5], [10, 2], [8, 9]
```

---

Step 5: Fitness Function

```python
def fitness(chromosome):
    max_depth, min_samples_split = chromosome
    model = DecisionTreeClassifier(max_depth=max_depth,
                                   min_samples_split=min_samples_split,
                                   random_state=42)
    scores = cross_val_score(model, X, y, cv=5)
    return scores.mean()
```

Meaning:

* The **fitness function** evaluates how “good” a chromosome is.
* Each chromosome’s genes (`max_depth` and `min_samples_split`) are used to train a `DecisionTreeClassifier`.
* `cross_val_score(..., cv=5)` performs **5-fold cross-validation**, splitting the dataset into 5 parts to get reliable accuracy.
* The mean accuracy is returned as the **fitness value**.

mathematical idea:

[
Fitness(C) = 1/k *  sumation{i=1}^{k} {Accuracy}_i
]
where ( k = 5 ) folds.

---

Step 6: Selection

```python
def selection(population, fitnesses):
    idx = np.argsort(fitnesses)[-2:]
    return [population[idx[0]], population[idx[1]]]
```

Meaning:

* Selects the **top 2 chromosomes (parents)** with highest fitness (accuracy).
* These two will reproduce to create the next generation.

---

Step 7: Crossover

```python
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1)-1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2
```

Meaning:

* **Single-point crossover**: choose a random split point and swap genes.

Example:

```
Parent1 = [5, 3]
Parent2 = [8, 7]
Crossover point = 1
→ Child1 = [5, 7]
→ Child2 = [8, 3]
```

So, genetic material (hyperparameters) is exchanged.

---

Step 8: Mutation

```python
def mutate(chromosome):
    if random.random() < MUTATION_RATE:
        chromosome[0] = random.randint(1, 20)
    if random.random() < MUTATION_RATE:
        chromosome[1] = random.randint(2, 10)
    return chromosome
```

Meaning:

* Introduces randomness (biological mutation).
* With 20% probability, a chromosome’s gene is replaced with a random value.
* Helps the algorithm **escape local optima** (i.e., prevents getting stuck at suboptimal results).

---

Step 9: Initialize Population

```python
population = [create_chromosome() for _ in range(POP_SIZE)]
```

Creates the **initial random population** of 20 chromosomes.

Example:

```
[[3, 5], [10, 2], [6, 7], [18, 9], ...]
```

---

Step 10: Run the Genetic Algorithm

```python
for gen in range(N_GENERATIONS):
    fitnesses = [fitness(chromo) for chromo in population]
    print(f"Generation {gen} - Best Fitness: {max(fitnesses):.4f}")

    new_population = []
    parents = selection(population, fitnesses)

    for _ in range(POP_SIZE // 2):
        child1, child2 = crossover(parents[0], parents[1])
        new_population.append(mutate(child1))
        new_population.append(mutate(child2))

    population = new_population
```

Meaning (step-by-step):

1. **Compute fitness** of each chromosome.
2. **Display best fitness** of that generation.
3. **Select top 2 parents**.
4. **Perform crossover and mutation** to create children.
5. **Replace old population** with new generation.

This process repeats for 10 generations, simulating *evolution*.

---

Step 11: Best Hyperparameters

```python
fitnesses = [fitness(chromo) for chromo in population]
best_idx = np.argmax(fitnesses)
print("\nBest Hyperparameters:", population[best_idx])
print("Best Accuracy:", fitnesses[best_idx])
```

Finds the **best chromosome** (highest accuracy) after all generations.

---

Output Explanation

Given Output:

```
Generation 0 - Best Fitness: 0.9733
Generation 1 - Best Fitness: 0.9733
Generation 2 - Best Fitness: 0.9733
Generation 3 - Best Fitness: 0.9733
Generation 4 - Best Fitness: 0.9733
Generation 5 - Best Fitness: 0.9733
Generation 6 - Best Fitness: 0.9733
Generation 7 - Best Fitness: 0.9733
Generation 8 - Best Fitness: 0.9733
Generation 9 - Best Fitness: 0.9733

Best Hyperparameters: [3, 9]
Best Accuracy: 0.9733333333333334
```

Interpretation:

* **Generation 0 → 9:**
  The best accuracy (`fitness`) remains **0.9733**.
  This means from the very first generation, a nearly optimal solution was already found.

* **Best Hyperparameters:** `[3, 9]`

  * `max_depth = 3` (tree depth = 3 levels)
  * `min_samples_split = 9` (a node splits only if ≥9 samples)

* **Best Accuracy:** `0.9733` (≈ 97.33%)

  * This is the mean cross-validation accuracy across 5 folds.

---

Why Accuracy Did Not Improve?

1. The dataset might be **small or well-structured**, so early solutions were already optimal.
2. GA population (20) and generations (10) are small — but still sufficient for a simple parameter space.
3. Mutation and crossover may not find better combinations since `[3, 9]` already gives near-perfect accuracy.

---

Concept Summary Diagram

```
┌──────────────────────────────────────────────┐
│                Genetic Algorithm             │
├──────────────────────────────────────────────┤
│ 1. Initialize random population              │
│ 2. Evaluate fitness (Decision Tree Accuracy) │
│ 3. Select best parents                       │
│ 4. Crossover to create children              │
│ 5. Mutate some children                      │
│ 6. Form new population                       │
│ 7. Repeat for N generations                  │
└──────────────────────────────────────────────┘
```

---

Final Summary

| Step                  | Function                                                   | Purpose |
| --------------------- | ---------------------------------------------------------- | ------- |
| `create_chromosome()` | Creates a random solution `[max_depth, min_samples_split]` |         |
| `fitness()`           | Measures model accuracy via cross-validation               |         |
| `selection()`         | Chooses top 2 best-performing parents                      |         |
| `crossover()`         | Mixes genes of parents to produce offspring                |         |
| `mutate()`            | Randomly changes genes to maintain diversity               |         |
| GA loop               | Evolves population to maximize accuracy                    |         |
| Output                | Displays best hyperparameters and their accuracy           |         |

---

A Genetic Algorithm (GA) is a nature-inspired optimization technique based on the principles of evolutionary biology and natural selection. The idea was first introduced by John Holland in the 1970s and has since been widely adopted for solving both simple and complex optimization problems. Unlike conventional optimization techniques, GA does not require gradient information or mathematical properties of the problem, making it suitable for non-linear, high-dimensional, and discontinuous search spaces.
At its core, GA imitates the process of Darwinian evolution, where stronger individuals in a population are more likely to survive and pass on their traits to the next generation. This is applied in computational systems to iteratively evolve solutions to optimization problems.

---
1. Representation (Chromosomes)

Each potential solution is encoded as a chromosome, often represented as a binary string, integer, or real-valued vector. For example, in machine learning hyperparameter tuning, a chromosome might encode values such as:

Learning rate = 0.01

Number of hidden layers = 3

Regularization parameter = 0.1

---
2. Population Initialization

The algorithm starts with a population of randomly generated chromosomes. The population size is an important parameter, as it determines the diversity of potential solutions in the search space. A larger population increases exploration but requires more computation.

---

3. Fitness Evaluation

Each chromosome is evaluated using a fitness function, which measures the quality of the solution. In the case of machine learning, this may correspond to the accuracy, precision, recall, or mean squared error of the model trained with the given parameters. The better the model performs, the higher the fitness value of its corresponding chromosome.

---

4. Selection

The next step is to select parents from the population based on their fitness. Higher fitness solutions have a greater probability of being selected. Common strategies include:

Roulette Wheel Selection (Proportionate Selection): Probability proportional to fitness.

Tournament Selection: A subset of chromosomes competes, and the best is selected.

Rank Selection: Chromosomes are ranked, and probabilities are assigned accordingly.

This ensures that good solutions are more likely to reproduce while maintaining diversity.

---

5. Crossover (Recombination)

Selected parents undergo crossover to produce offspring by exchanging parts of their chromosomes. This mimics biological reproduction and helps combine the strengths of two solutions. Types of crossover include:

Single-Point Crossover: A crossover point is chosen, and segments are swapped.

Two-Point Crossover: Two crossover points are chosen.

Uniform Crossover: Each gene is swapped with some probability.

This mechanism introduces new solutions that can potentially perform better than their parents.

---

6. Mutation

To maintain diversity in the population and prevent premature convergence, GA introduces small random changes to chromosomes. For example:

Flipping a bit in a binary string.

Slightly altering a real-valued parameter.


Mutation ensures that the search process explores new regions of the solution space.

---

7. Replacement (Survivor Selection)

After generating offspring, the new generation is formed by replacing some or all members of the old population. Strategies include:

Generational Replacement: Entire population replaced.

Elitism: Best individuals from the previous generation are carried forward.

---
8. Termination Criteria

The GA continues until a stopping condition is met, such as:
A maximum number of generations.

No significant improvement in fitness.

Achieving a desired accuracy or error threshold.

---
Application in Machine Learning

In machine learning, selecting optimal hyperparameters is often challenging due to the curse of dimensionality and non-linear interactions between parameters. Traditional methods like grid search or random search are computationally expensive and often miss good solutions.

By contrast, GA offers the following advantages:
Exploration: Searches a wide space of possible parameter values.
Adaptation: Learns from previous generations to improve solutions.
Efficiency: Finds near-optimal solutions with fewer evaluations than exhaustive search.

For instance, in optimizing a Support Vector Machine (SVM):
The chromosome may encode kernel type, C, and gamma.
GA evaluates each combination using cross-validation accuracy.
Over generations, the algorithm converges toward the best set of parameters.

---
Conclusion:

This assignment demonstrates how Genetic Algorithms can optimize machine learning model parameters efficiently. GA provides a robust, adaptive, and intelligent search strategy for hyperparameter tuning, outperforming conventional methods like grid search in terms of flexibility and scalability.
By applying GA, the machine learning model achieves higher accuracy and generalization, proving the effectiveness of evolutionary computing in modern AI applications.

Sure ✅ — here’s the **Genetic Algorithm (GA)** written in a clear **algorithmic format** suitable for your **SCOA assignment** (with steps, pseudocode, and explanation).

---

**Genetic Algorithm (GA)**

---

**Algorithm Steps**

**Step 1: Initialization**

1. Define the **population size (POP_SIZE)** and **number of generations (N_GENERATIONS)**.
2. Define the **chromosome structure** — each chromosome represents one possible solution.
   Example:
   [
   Chromosome = [{max_depth}, {min_samples_split}]
   ]
3. Randomly initialize the population with possible values.

---

### **Step 2: Fitness Evaluation**

1. For each chromosome in the population:

   * Decode its genes into actual parameters.
   * Evaluate its **fitness** using a performance metric (e.g., model accuracy).
     [
     Fitness(C) ={Mean Cross-Validation Accuracy}
     ]
2. Store all fitness values.

---

### **Step 3: Selection**

1. Select the best-performing individuals (parents) based on fitness.
2. Common methods:

   * **Roulette Wheel Selection**
   * **Tournament Selection**
   * **Elitism (Top-k selection)**

---

### **Step 4: Crossover (Recombination)**

1. Randomly choose a **crossover point** between parent genes.
2. Exchange genetic material between parents to produce two children.

   Example:

   ```
   Parent1 = [5, 3]
   Parent2 = [8, 7]
   Crossover point = 1
   → Child1 = [5, 7]
   → Child2 = [8, 3]
   ```

---

### **Step 5: Mutation**

1. For each child, with a small probability (**MUTATION_RATE**), change one or more genes randomly.
   [
   {If random() < mutation_rate → change gene to new random value}
   ]
2. This helps maintain diversity and prevent premature convergence.

---

### **Step 6: New Generation**

1. Combine all children to form the **new population**.
2. Repeat Steps 2–5 for each generation.

---

### **Step 7: Termination**

1. Stop when the **maximum number of generations** is reached or the **fitness value converges** (no improvement).
2. The **best chromosome** represents the optimal solution.

---

## **Pseudocode of Genetic Algorithm**

```
BEGIN
  Initialize population P with random chromosomes
  FOR generation = 1 to N_GENERATIONS DO
      FOR each chromosome Ci in P DO
          Compute fitness(Ci)
      END FOR

      Select best parents (P1, P2) from P
      Create empty new_population

      WHILE size(new_population) < POP_SIZE DO
          Perform crossover on (P1, P2) → (Child1, Child2)
          Mutate(Child1)
          Mutate(Child2)
          Add Child1, Child2 to new_population
      END WHILE

      Replace P ← new_population
      Print best fitness of current generation
  END FOR

  Return chromosome with highest fitness as optimal solution
END
```

---

## **Step-by-Step Example in Context (Decision Tree Tuning)**

| Step | Description                                                       |
| ---- | ----------------------------------------------------------------- |
| 1    | Randomly create 20 chromosomes like `[3, 5]`, `[10, 2]`, `[6, 8]` |
| 2    | Train Decision Tree with these parameters and compute accuracy    |
| 3    | Select the top 2 performing chromosomes                           |
| 4    | Perform crossover and mutation to generate new children           |
| 5    | Replace old population with new one                               |
| 6    | Repeat for 10 generations                                         |
| 7    | Return the best parameters, e.g., `[3, 9]` with accuracy 0.9733   |

---

## **Mathematical Representation**

1. **Fitness Function:**
   [
   F(C_i) = 1/k sumation{j=1}^{k} {Accuracy}_{ij}
   ]
   where ( k ) = number of cross-validation folds.

2. **Selection Probability:**
   [
   P_i = {F(C_i)} / {sumation{j=1}^{N} F(C_j)}
   ]

3. **Crossover Operation:**
   [
   {Child1} = P1[0:p] + P2[p:]
   ]
   [
   {Child2} = P2[0:p] + P1[p:]
   ]

4. **Mutation Operation:**
   [
   {If random() < rate, then } gene_i = random_value()
   ]

---

## **Advantages of GA**

* Works well for **nonlinear**, **complex**, or **multi-modal** problems.
* Does **not require gradient** information.
* Can escape **local optima** due to mutation.

---

## **Disadvantages**

* Computationally expensive for large populations.
* Randomness may slow convergence.
* May require tuning of GA parameters (population size, mutation rate, etc.).

---

## **Output Example**

```
Generation 0 - Best Fitness: 0.9733
Generation 1 - Best Fitness: 0.9733
...
Generation 9 - Best Fitness: 0.9733

Best Hyperparameters: [3, 9]
Best Accuracy: 0.9733
```

This means the GA converged quickly to the best solution where:

* `max_depth = 3`
* `min_samples_split = 9`
* Average accuracy = **97.33%**

---
